# Explaining random forest model predictions with Shapley values

Shapley values provide an estimate of how much any particular feature influences the model decision. When Shapley values are averaged they provide a measure of the overall influence of a feature.

Shapley values may be used across model types, and so provide a model-agnostic measure of a feature’s influence. This means that the influence of features may be compared across model types, and it allows black box models like neural networks to be explained, at least in part.

For more on Shapley values in general see Chris Molner’s excellent book chapter:

https://christophm.github.io/interpretable-ml-book/shapley.html


More information on the shap library, inclusiong lots of useful examples may be found at: https://shap.readthedocs.io/en/latest/index.html

Here we provide an example of using shap with random forests.

## Note on shap version 0.40:

There is a bug in the waterfall plot where `show=False` (required to save plot) fails. To correct this find the `_waterfall.py` file in the shap library (e.g. in `anaconda/envs/samuel2/lib/python3.8/site-packages/shap/plots`, and search/replace all `pl.` to `.plt`.

## Load packages

In [1]:
# Turn warnings off to keep notebook tidy
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import shap

from sklearn.ensemble import RandomForestClassifier

## Load data

We will use the first train/test split.

In [2]:
data_loc = '../../data/sam_1/kfold_5fold/'

train = pd.read_csv(data_loc + 'train_0.csv')
test = pd.read_csv(data_loc + 'test_0.csv')

## Fit random forest model

Fit model and get feature importances.

In [3]:
# Get X and y
X_train = train.drop('S2Thrombolysis', axis=1)
X_test = test.drop('S2Thrombolysis', axis=1)
y_train = train['S2Thrombolysis']
y_test = test['S2Thrombolysis']

# One hot encode hospitals
X_train_hosp = pd.get_dummies(X_train['StrokeTeam'], prefix = 'team')
X_train = pd.concat([X_train, X_train_hosp], axis=1)
X_train.drop('StrokeTeam', axis=1, inplace=True)
X_test_hosp = pd.get_dummies(X_test['StrokeTeam'], prefix = 'team')
X_test = pd.concat([X_test, X_test_hosp], axis=1)
X_test.drop('StrokeTeam', axis=1, inplace=True)

# Define and Fit model
model = RandomForestClassifier(
    n_estimators=100, n_jobs=-1, class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

# Get feature weights
features = list(X_train)
feature_importances = model.feature_importances_
importances = pd.DataFrame(index=features)
importances['importance'] = feature_importances
importances['rank'] = importances['importance'].rank(ascending=False).values

# Get predicted class and ptrobability
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

# Measure accuracy
accuracy = np.mean(y_pred == y_test)
print(f'Accuracy: {accuracy:0.3f}')

Accuracy: 0.845


## Shap values

### Get Shap values

We will get the Shap values for the test set. Can chose to calculate shap and save, or load shap explainer.

In [ ]:
calculate_shap_values = True

if calculate_shap_values:

    # Set up explainer using typical feature values from training set
    # Note: Use a sample of 100-1000 for this if explainer too slow
    explainer = shap.TreeExplainer(
        model, X_train.sample(100), model_output="probability")

    # Get Shapley values along with base and features
    shap_values_extended = explainer(X_test)
    # Shap values exist for each classification in a Tree; 1=give thrombolysis
    shap_values = shap_values_extended.values[:,:,1]
    
    # Save using pickle
    filename = './output/shap_values_extended_rf1.p'
    with open(filename, 'wb') as filehandler:
        pickle.dump(shap_values_extended, filehandler)
        
else:
    # Load preloaded explainer
    filename = './output/shap_values_extended_rf1.p'
    with open(filename, 'rb') as filehandler: 
        shap_values_extended = pickle.load(filehandler)
        shap_values = shap_values_extended.values[:,:,1]

 44%|=========           | 15525/35518 [42:27<54:40]       

In [ ]:
# Get mean Shap values for each feature
shap_values_mean = pd.DataFrame(index=features)
shap_values_mean['mean_shap'] = np.mean(shap_values, axis=0)
shap_values_mean['abs_mean_shap'] = np.abs(shap_values_mean)
shap_values_mean['mean_abs_shap'] = np.mean(np.abs(shap_values), axis=0)
shap_values_mean['rank'] = shap_values_mean['mean_abs_shap'].rank(
    ascending=False).values

### Compare top 10 weights and Shap values

In [ ]:
top_10_importances = \
    importances.sort_values('importance', ascending=False).head(10)
top_10_importances

When looking for the most influential Shap values we use the mean of the absolute Shap values for each feature.

In [ ]:
top_10_shap = shap_values_mean.sort_values(
    'mean_abs_shap', ascending=False).head(10)
top_10_shap

Get intersection between top 10 weights and top 10 Shap values.

In [ ]:
intersection = list(top_10_importances.index.intersection(top_10_shap.index))
print(f'Number of intersection values = {len(intersection)}')
print('\nIntersecting values:')
intersection

### Plot average Shap values against average weights

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
ax.scatter(importances['importance'], shap_values_mean['mean_abs_shap'])
ax.grid()
ax.set_xlabel('Feature importance')
ax.set_ylabel('Feature shap')
plt.show()

### Plot most influential features

This is a plot of the mean absolute Shap values.

In [ ]:
fig = shap.summary_plot(shap_values, X_test, plot_type='bar', max_display=12, show=False)
plt.savefig('output/shap_rf.jpg', dpi=300, bbox_inches='tight', pad_inches=0.2)
plt.show()

### Beeswarm plot

A Beeswarm plot shows all points. The feature value for each point is shown by the colour, and its position indicates the Shap value for that instance.

In [ ]:
fig = plt.figure(figsize=(6,6))

shap.summary_plot(shap_values=shap_values, 
                  features=X_test,
                  feature_names=features,
                  max_display=12,
                  cmap=plt.get_cmap('nipy_spectral'), show=False)
plt.savefig('output/rf_beeswarm.jpg', dpi=300, bbox_inches='tight', 
            pad_inches=0.2)
plt.show()

### Plot Waterfall and decision plot plots for instances with low or high probability of receiving thrombolysis

Waterfall plot and decision plots are alternative ways of plotting the influence of features for individual cases.

In [ ]:
# Get the location of an example each where probability of giving thrombolysis
# is <0.1 or >0.9

location_low_probability = np.where(y_prob < 0.1)[0][0]
location_high_probability = np.where(y_prob > 0.9)[0][0]

An example with low probability of receiving thrombolysis.

In [ ]:
fig = shap.plots.waterfall(shap_values_extended[location_low_probability][:,1],
                           show=False, max_display=12)
plt.savefig('output/rf_waterfall_low.jpg', dpi=300, bbox_inches='tight',
            pad_inches=0.2)
plt.show()

In [ ]:
instance = location_low_probability

chosen_instance = X_test.iloc[instance]
shap_values_instance = shap_values[instance]
expected_value = shap_values_extended.base_values[0][1]#explainer.expected_value
print(f"The base value is {expected_value:0.2f}")

shap.decision_plot(expected_value, shap_values_instance, chosen_instance)

An example with high probability of receiving thrombolysis.

In [ ]:
fig = shap.plots.waterfall(shap_values_extended[location_high_probability][:,1],
                           show=False, max_display=12)
plt.savefig('output/rf_waterfall_high.jpg', dpi=300, bbox_inches='tight',
            pad_inches=0.2)
plt.show()

In [ ]:
instance = location_high_probability

chosen_instance = X_test.iloc[instance]
shap_values_instance = shap_values[instance]
expected_value = shap_values_extended.base_values[0][1]#explainer.expected_value
print(f"The base value is {expected_value:0.2f}")

shap.decision_plot(expected_value, shap_values_instance, chosen_instance)

### Show the relationship between feature value and Shap value for top 5 influential features.

In [ ]:
feat_to_show = top_10_shap.index[0:5]

for feat in feat_to_show:
    shap.plots.scatter(shap_values_extended[:, feat][:,1], x_jitter=0)

Examine `S2BrainImagingTime_min` in range 0-400 minutes.

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
shap.plots.scatter(shap_values_extended[:, 'S2BrainImagingTime_min'][:,1], 
                   x_jitter=0, ax=ax, show=False)
ax.set_xlim(0,400)